In [79]:
import os
import json
from dotenv import load_dotenv
from tqdm.notebook import tqdm

load_dotenv()  # take environment variables from .env.

True

In [3]:
# establish a connection to the MongoDB database
from pymongo import MongoClient

# connect to your Atlas cluster
client = MongoClient(os.environ["MONGODB_URI"])

In [30]:
# establish a connection to the PostgreSQL database
import psycopg2 as pg

conn = pg.connect(
    dbname=os.environ["POSTGRES_DB"],
    user=os.environ["POSTGRES_USER"],
    password=os.environ["POSTGRES_PASSWORD"],
    host=os.environ["POSTGRES_HOST"]
)
cursor = conn.cursor()

# LangChain: Models, Prompts and Output Parsers


## Outline

 * Direct API calls to OpenAI
 * API calls through LangChain:
   * Prompts
   * Models
   * Output parsers

In [83]:
import openai

openai.api_key = os.environ['OPENAI_APIKEY']
llm_model = "gpt-4o-mini"
# llm_model = "chatgpt-4o-latest"


In [84]:
from langchain.chat_models import ChatOpenAI

In [85]:
# To control the randomness and creativity of the generated
# text by an LLM, use temperature = 0.0
chat = ChatOpenAI(temperature=0.0, model=llm_model, openai_api_key=openai.api_key)


### Clean up class names

In [86]:
template_string_classes = """
You are a highly skilled OWL (Web Ontology Language) ontology engineer. Your task is to assist in the creation, validation, and optimization of OWL ontologies, which are formal representations of knowledge. You have expert knowledge in knowledge engineering, description logic, and semantic web technologies. You also excel at defining classes, properties, and relationships between entities, ensuring logical consistency, and facilitating the sharing of knowledge across different domains.

An OWL ontology is a structured framework used to represent and share knowledge about a particular domain. It consists of classes (concepts), properties (relationships and attributes), and individuals (instances). OWL ontologies allow for the modeling of rich, complex relationships between data in a machine-readable format, enabling advanced reasoning, querying, and inference over that data.
Core OWL Concepts:

    Classes: These represent sets or collections of individuals, typically abstract concepts or types (e.g., "Character," "Weapon," "GameLevel").
    Individuals: Instances of classes (e.g., "Mario" is an individual of the class "Character"; "Sword of Flames" is an individual of the class "Weapon").
    Object Properties: Define relationships between two individuals (e.g., "wieldsWeapon" linking a character to a weapon they use, or "locatedIn" linking a character to a particular game level).
    Datatype Properties: Define relationships between an individual and a data value (e.g., "hasHealthPoints" linking a character to a numeric value representing their health).
    SubClassOf: A relation where one class is a subclass of another, inheriting properties (e.g., "BossCharacter" is a subclass of "Character").
    Equivalence: Used to state that two classes or properties are equivalent (e.g., "MagicWeapon" may be declared equivalent to "SpecialWeapon").
    Disjoint Classes: These are classes that cannot share instances (e.g., "Weapon" and "ConsumableItem" are disjoint classes, meaning an item cannot be both a weapon and a consumable).

OWL Inference and Reasoning:

One of the powerful aspects of OWL is that it allows for reasoning over data. Inference engines can deduce new facts based on the relationships and properties defined in the ontology. For example, if "BossCharacter" is a subclass of "Character" and "Bowser" is an individual of "BossCharacter," it can be inferred that "Bowser" is also an individual of "Character." Additionally, if a property like "wieldsWeapon" is defined, you could infer that "Bowser wields a FireballWeapon" if such an individual and relationship are defined.

Your role is organize and structure a list of classes derived from web data. Classes might be the same but have a slightly different spelling, capitalization, wording, abbreviation or plurality. First, you need to clean up these classes and provide a list of final classes. The final classes should be unique.

Second, you need merge some classes in a single class if that makes ontological sense. For example, GameLevel and Level are the same class and should be merged into a single class. All of the class that have In-Game- , or similar, prefix should drop the prefix and be merged with the class without the prefix.

You are given a list of classes as input. You need to provide a list of tuples as output: ("original class name", "cleaned up class name"). For example, if the input is ["Apple", "apple", "Banana", "Ba-Na-Na", "Fruit"], the output should be [("Apple", "Apple"), ("apple", "Apple"), ("Banana", "Banana"), ("Ba-Na-Na", "Banana"), ("Fruit", "Fruit")].

Original classes are case-insensitive. Cleaned up classes should be capitalized and have no special characters or spaces.

Input classes: ```{classes}```

"""

In [69]:
from langchain.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_template(template_string_classes)

In [70]:
prompt_template.messages[0].prompt

PromptTemplate(input_variables=['classes'], input_types={}, partial_variables={}, template='\nYou are a highly skilled OWL (Web Ontology Language) ontology engineer. Your task is to assist in the creation, validation, and optimization of OWL ontologies, which are formal representations of knowledge. You have expert knowledge in knowledge engineering, description logic, and semantic web technologies. You also excel at defining classes, properties, and relationships between entities, ensuring logical consistency, and facilitating the sharing of knowledge across different domains.\n\nAn OWL ontology is a structured framework used to represent and share knowledge about a particular domain. It consists of classes (concepts), properties (relationships and attributes), and individuals (instances). OWL ontologies allow for the modeling of rich, complex relationships between data in a machine-readable format, enabling advanced reasoning, querying, and inference over that data.\nCore OWL Concept

In [71]:
prompt_template.messages[0].prompt.input_variables

['classes']

In [80]:
cursor.execute("SELECT distinct target_class FROM fandom_properties_clean where target_class not in (select class from fandom_classes) ORDER BY target_class")
classes_list = cursor.fetchall()

# classes_1 = '\n'.join([c[0] for c in classes_list[:2000]])
# classes_2 = '\n'.join([c[0] for c in classes_list[2000:]])
classes = '\n'.join([c[0] for c in classes_list])

In [81]:
len(classes_list)

1

In [82]:
classes_list

[('',)]

In [74]:
fandom_input = prompt_template.format_messages(classes=classes)

In [75]:
# Call the LLM to translate to the style of the customer message
entity_info = chat.invoke(fandom_input)

In [76]:
print(entity_info.content)

To clean up and organize the list of classes, we need to standardize the naming conventions, remove any prefixes like "In-Game-", and merge similar classes. Below is a list of tuples showing the original class names and their cleaned-up versions:

```python
[
    ("Ability", "Ability"),
    ("AbilityCost", "AbilityCost"),
    ("AbilityDamage", "AbilityDamage"),
    ("AbilityDescription", "AbilityDescription"),
    ("AbilityEffect", "AbilityEffect"),
    ("AbilityID", "AbilityID"),
    ("AbilityLevel", "AbilityLevel"),
    ("AbilityPower", "AbilityPower"),
    ("AbilitySource", "AbilitySource"),
    ("AbilitySubtype", "AbilitySubtype"),
    ("AbilityText", "AbilityText"),
    ("AbilityTree", "AbilityTree"),
    ("AbilityType", "AbilityType"),
    ("Absorption", "Absorption"),
    ("AbundanceLevel", "AbundanceLevel"),
    ("AbundanceType", "AbundanceType"),
    ("Acceleration", "Acceleration"),
    ("AccelerationRating", "AccelerationRating"),
    ("Accent", "Accent"),
    ("AccessCardTy

In [42]:
(entity_info.content[:-5] + "]").split('```python')[1]

'\n[\n    ("ability", "Ability"),\n    ("Ability", "Ability"),\n    ("AbilityCost", "AbilityCost"),\n    ("Ability Cost", "AbilityCost"),\n    ("AbilityDamage", "AbilityDamage"),\n    ("AbilityDescription", "AbilityDescription"),\n    ("Ability Description", "AbilityDescription"),\n    ("AbilityEffect", "AbilityEffect"),\n    ("Ability Effect", "AbilityEffect"),\n    ("AbilityID", "AbilityID"),\n    ("Ability Level", "AbilityLevel"),\n    ("AbilityPower", "AbilityPower"),\n    ("Ability Source", "AbilitySource"),\n    ("AbilitySubtype", "AbilitySubtype"),\n    ("AbilityText", "AbilityText"),\n    ("AbilityTree", "AbilityTree"),\n    ("Ability Tree", "AbilityTree"),\n    ("ability_type", "AbilityType"),\n    ("AbilityType", "AbilityType"),\n    ("Absorption", "Absorption"),\n    ("AbundanceLevel", "AbundanceLevel"),\n    ("Abundance Type", "AbundanceType"),\n    ("acceleration", "Acceleration"),\n    ("Acceleration", "Acceleration"),\n    ("Acceleration Rating", "AccelerationRating"),\n

In [77]:
# output = json.loads((entity_info.content[:-5] + "]").split('```python')[1])
output = [
    ("Ability", "Ability"),
    ("AbilityCost", "AbilityCost"),
    ("AbilityDamage", "AbilityDamage"),
    ("AbilityDescription", "AbilityDescription"),
    ("AbilityEffect", "AbilityEffect"),
    ("AbilityID", "AbilityID"),
    ("AbilityLevel", "AbilityLevel"),
    ("AbilityPower", "AbilityPower"),
    ("AbilitySource", "AbilitySource"),
    ("AbilitySubtype", "AbilitySubtype"),
    ("AbilityText", "AbilityText"),
    ("AbilityTree", "AbilityTree"),
    ("AbilityType", "AbilityType"),
    ("Absorption", "Absorption"),
    ("AbundanceLevel", "AbundanceLevel"),
    ("AbundanceType", "AbundanceType"),
    ("Acceleration", "Acceleration"),
    ("AccelerationRating", "AccelerationRating"),
    ("Accent", "Accent"),
    ("AccessCardType", "AccessCardType"),
    ("AccessMethod", "AccessMethod"),
    ("Accessory", "Accessory"),
    ("AccessPoint", "AccessPoint"),
    ("AccessType", "AccessType"),
    ("Account", "Account"),
    ("AccountType", "AccountType"),
    ("Accuracy", "Accuracy"),
    ("AccuracyLevel", "AccuracyLevel"),
    ("AccuracyRating", "AccuracyRating"),
    ("AccuracyValue", "AccuracyValue"),
    ("Achievement", "Achievement"),
    ("AchievementDescription", "AchievementDescription"),
    ("AchievementPoints", "AchievementPoints"),
    ("AchievementRequirement", "AchievementRequirement"),
    ("AchievementTask", "AchievementTask"),
    ("AchievementType", "AchievementType"),
    ("AcquirementMethod", "AcquirementMethod"),
    ("AcquirementType", "AcquirementType"),
    ("Acquisition", "Acquisition"),
    ("AcquisitionCost", "AcquisitionCost"),
    ("AcquisitionMethod", "AcquisitionMethod"),
    ("AcquisitionStatus", "AcquisitionStatus"),
    ("AcquisitionType", "AcquisitionType"),
    ("Act", "Act"),
    ("Action", "Action"),
    ("ActionDescription", "ActionDescription"),
    ("ActionInput", "ActionInput"),
    ("ActionPoint", "ActionPoint"),
    ("ActionPoints", "ActionPoints"),
    ("ActionType", "ActionType"),
    ("Activation", "Activation"),
    ("ActivationCondition", "ActivationCondition"),
    ("ActivationMode", "ActivationMode"),
    ("ActivationState", "ActivationState"),
    ("ActivationType", "ActivationType"),
    ("ActiveEffect", "ActiveEffect"),
    ("ActiveModules", "ActiveModules"),
    ("Activity", "Activity"),
    ("ActivityType", "ActivityType"),
    ("Actor", "Actor"),
    ("Actress", "Actress"),
    ("Addiction", "Addiction"),
    ("Addition", "Addition"),
    ("Address", "Address"),
    ("AdminProperties", "AdminProperties"),
    ("AdminType", "AdminType"),
    ("AdvancedClass", "AdvancedClass"),
    ("AdvancedMetric", "AdvancedMetric"),
    ("Advantage", "Advantage"),
    ("AdventureModule", "AdventureModule"),
    ("Advice", "Advice"),
    ("AdviceText", "AdviceText"),
    ("Aesthetic", "Aesthetic"),
    ("AestheticType", "AestheticType"),
    ("Affiliation", "Affiliation"),
    ("Affinity", "Affinity"),
    ("Affliction", "Affliction"),
    ("Age", "Age"),
    ("AgeCategory", "AgeCategory"),
    ("AgeComparison", "AgeComparison"),
    ("AgeGroup", "AgeGroup"),
    ("Agent", "Agent"),
    ("AgeRange", "AgeRange"),
    ("AgeRating", "AgeRating"),
    ("AgeStatus", "AgeStatus"),
    ("AgeValue", "AgeValue"),
    ("Aggression", "Aggression"),
    ("AggressionLevel", "AggressionLevel"),
    ("AggroRange", "AggroRange"),
    ("Agility", "Agility"),
    ("AgilityStat", "AgilityStat"),
    ("AgilityValue", "AgilityValue"),
    ("Ailment", "Ailment"),
    ("Aim", "Aim"),
    ("AimModifier", "AimModifier"),
    ("AirAttack", "AirAttack"),
    ("AirDate", "AirDate"),
    ("AirDefense", "AirDefense"),
    ("AirRotation", "AirRotation"),
    ("Album", "Album"),
    ("Alias", "Alias"),
    ("Alignment", "Alignment"),
    ("Allegiance", "Allegiance"),
    ("AllegianceType", "AllegianceType"),
    ("Alliance", "Alliance"),
    ("AllianceType", "AllianceType"),
    ("Alteration", "Alteration"),
    ("AlternateName", "AlternateName"),
    ("AlternativeMode", "AlternativeMode"),
    ("AlternativeName", "AlternativeName"),
    ("Altitude", "Altitude"),
    ("Ambition", "Ambition"),
    ("Ammo", "Ammo"),
    ("AmmoCapacity", "AmmoCapacity"),
    ("AmmoCount", "AmmoCount"),
    ("AmmoQuantity", "AmmoQuantity"),
    ("AmmoReloaded", "AmmoReloaded"),
    ("AmmoState", "AmmoState"),
    ("AmmoStatus", "AmmoStatus"),
    ("AmmoType", "AmmoType"),
    ("AmmoValue", "AmmoValue"),
    ("Ammunition", "Ammunition"),
    ("AmmunitionCapacity", "AmmunitionCapacity"),
    ("AmmunitionRange", "AmmunitionRange"),
    ("AmmunitionType", "AmmunitionType"),
    ("Amount", "Amount"),
    ("Ancestry", "Ancestry"),
    ("Angle", "Angle"),
    ("Animal", "Animal"),
    ("AnimalCount", "AnimalCount"),
    ("AnimationEffect", "AnimationEffect"),
    ("AnimationState", "AnimationState"),
    ("Animator", "Animator"),
    ("Animatronic", "Animatronic"),
    ("Anime", "Anime"),
    ("AnimeEpisode", "AnimeEpisode"),
    ("AnimeEvent", "AnimeEvent"),
    ("Appearance", "Appearance"),
    ("AppearanceAttribute", "AppearanceAttribute"),
    ("AppearanceCode", "AppearanceCode"),
    ("AppearanceCount", "AppearanceCount"),
    ("AppearanceDescription", "AppearanceDescription"),
    ("AppearanceDetail", "AppearanceDetail"),
    ("AppearanceID", "AppearanceID"),
    ("AppearanceLocation", "AppearanceLocation"),
    ("AppearanceSchedule", "AppearanceSchedule"),
    ("AppearanceTicker", "AppearanceTicker"),
    ("Arcana", "Arcana"),
    ("Archetype", "Archetype"),
    ("Architecture", "Architecture"),
    ("Area", "Area"),
    ("AreaCode", "AreaCode"),
    ("AreaEffect", "AreaEffect"),
    ("AreaLevel", "AreaLevel"),
    ("AreaMeasure", "AreaMeasure"),
    ("AreaMeasurement", "AreaMeasurement"),
    ("AreaOfEffect", "AreaOfEffect"),
    ("AreaType", "AreaType"),
    ("Arm", "Arm"),
    ("Armament", "Armament"),
    ("ArmedForces", "ArmedForces"),
    ("Armor", "Armor"),
    ("ArmorDistribution", "ArmorDistribution"),
    ("ArmorPenetrationValue", "ArmorPenetrationValue"),
    ("ArmorPoints", "ArmorPoints"),
    ("ArmorRating", "ArmorRating"),
    ("ArmorSetBonus", "ArmorSetBonus"),
    ("ArmorType", "ArmorType"),
    ("ArmorValue", "ArmorValue"),
    ("ArmyGroup", "ArmyGroup"),
    ("Arranger", "Arranger"),
    ("Art", "Art"),
    ("Artifact", "Artifact"),
    ("Artist", "Artist"),
    ("Artwork", "Artwork"),
    ("ASIN", "ASIN"),
    ("AspectRatio", "AspectRatio"),
    ("Assignment", "Assignment"),
    ("Assist", "Assist"),
    ("AssistType", "AssistType"),
    ("Asteroid", "Asteroid"),
    ("AstralSign", "AstralSign"),
    ("AtomicNumber", "AtomicNumber"),
    ("Attachment", "Attachment"),
    ("Attack", "Attack"),
    ("AttackDamage", "AttackDamage"),
    ("AttackDamageValue", "AttackDamageValue"),
    ("AttackDelay", "AttackDelay"),
    ("AttackDirection", "AttackDirection"),
    ("AttackLimit", "AttackLimit"),
    ("AttackMethod", "AttackMethod"),
    ("AttackMode", "AttackMode"),
    ("AttackMove", "AttackMove"),
    ("AttackPerSecond", "AttackPerSecond"),
    ("AttackPoints", "AttackPoints"),
    ("AttackPower", "AttackPower"),
    ("AttackRange", "AttackRange"),
    ("AttackRate", "AttackRate"),
    ("AttackSpeed", "AttackSpeed"),
    ("AttacksPerSecond", "AttacksPerSecond"),
    ("AttackStat", "AttackStat"),
    ("AttackStrategy", "AttackStrategy"),
    ("AttackStrength", "AttackStrength"),
    ("AttackStyle", "AttackStyle"),
    ("AttackTechnique", "AttackTechnique"),
    ("AttackType", "AttackType"),
    ("AttackValue", "AttackValue"),
    ("Attitude", "Attitude"),
    ("AttitudeType", "AttitudeType"),
    ("AttractBonus", "AttractBonus"),
    ("Attractiveness", "Attractiveness"),
    ("AttractivenessScore", "AttractivenessScore"),
    ("Attribute", "Attribute"),
    ("Attributes", "Attribute"),
    ("Audio", "Audio"),
    ("AudioAlbum", "AudioAlbum"),
    ("AudioFile", "AudioFile"),
    ("AudioTrack", "AudioTrack"),
    ("Augment", "Augment"),
    ("Augmentation", "Augmentation"),
    ("Author", "Author"),
    ("Autopsy", "Autopsy"),
    ("AutopsyProject", "AutopsyProject"),
    ("AuxiliaryEffect", "AuxiliaryEffect"),
    ("Availability", "Availability"),
    ("AvailabilityLevel", "AvailabilityLevel"),
    ("AvailabilityPeriod", "AvailabilityPeriod"),
    ("AvailabilityStatus", "AvailabilityStatus"),
    ("AvailabilityType", "AvailabilityType"),
    ("Aversion", "Aversion"),
    ("Award", "Award"),
    ("AWDType", "AWDType"),
    ("BackBling", "BackBling"),
    ("Background", "Background"),
    ("BackgroundMusic", "BackgroundMusic"),
    ("BarcodeNumber", "BarcodeNumber"),
    ("BarrelLength", "BarrelLength"),
    ("Base", "Base"),
    ("BaseLevel", "BaseLevel"),
    ("BaseStamina", "BaseStamina"),
    ("BaseValue", "BaseValue"),
    ("Battle", "Battle"),
    ("BattleEffect", "BattleEffect"),
    ("BattleFlag", "BattleFlag"),
    ("BattlePassAct", "BattlePassAct"),
    ("BattleRating", "BattleRating"),
    ("BattleRecord", "BattleRecord"),
    ("BattleType", "BattleType"),
    ("BeamDamage", "BeamDamage"),
    ("BeamRange", "BeamRange"),
    ("Beats", "Beats"),
    ("Bees", "Bees"),
    ("Behavior", "Behavior"),
    ("BehaviorType", "BehaviorType"),
    ("Behemoth", "Behemoth"),
    ("Belief", "Belief"),
    ("Benefit", "Benefit"),
    ("Beverage", "Beverage"),
    ("BGM", "BGM"),
    ("BGMusic", "BGMusic"),
    ("Binding", "Binding"),
    ("BindingType", "BindingType"),
    ("BiologicalCondition", "BiologicalCondition"),
    ("BiologicalDiversity", "BiologicalDiversity"),
    ("BiologicalFamily", "BiologicalFamily"),
    ("BiologicalKingdom", "BiologicalKingdom"),
    ("BiologicalOrder", "BiologicalOrder"),
    ("BiologicalState", "BiologicalState"),
    ("BiologicalStatus", "BiologicalStatus"),
    ("BiologicalTaxon", "BiologicalTaxon"),
    ("BiologyStatus", "BiologyStatus"),
    ("BiomassSample", "BiomassSample"),
    ("Biome", "Biome"),
    ("BiomeID", "BiomeID"),
    ("Biped", "Biped"),
    ("BirdSequence", "BirdSequence"),
    ("Birth", "Birth"),
    ("Birthdate", "Birthdate"),
    ("Birthday", "Birthday"),
    ("BirthInfo", "BirthInfo"),
    ("BirthLocation", "BirthLocation"),
    ("Birthplace", "Birthplace"),
    ("BirthStatus", "BirthStatus"),
    ("Block", "Block"),
    ("BlockAmount", "BlockAmount"),
    ("BlockPercent", "BlockPercent"),
    ("BlockType", "BlockType"),
    ("BloodColor", "BloodColor"),
    ("BloodType", "BloodType"),
    ("Blueprint", "Blueprint"),
    ("BoardMember", "BoardMember"),
    ("Body", "Body"),
    ("BodyMeasurement", "BodyMeasurement"),
    ("BodyPart", "BodyPart"),
    ("BodySlot", "BodySlot"),
    ("BodyStyle", "BodyStyle"),
    ("BodyType", "BodyType"),
    ("BondCount", "BondCount"),
    ("Bonus", "Bonus"),
    ("BonusAttribute", "BonusAttribute"),
    ("BonusAttributes", "BonusAttribute"),
    ("BonusChance", "BonusChance"),
    ("BonusDetails", "BonusDetails"),
    ("BonusEffect", "BonusEffect"),
    ("BonusType", "BonusType"),
    ("Boo", "Boo"),
    ("Book", "Book"),
    ("Boolean", "Boolean"),
    ("BooleanValue", "BooleanValue"),
    ("BoostPadConfiguration", "BoostPadConfiguration"),
    ("Boss", "Boss"),
    ("BossCharacter", "BossCharacter"),
    ("BottomClothing", "BottomClothing"),
    ("BottomClothingColor", "BottomClothingColor"),
    ("BottomClothingType", "BottomClothingType"),
    ("Bounty", "Bounty"),
    ("BountyHistory", "BountyHistory"),
    ("BPM", "BPM"),
    ("Braking", "Braking"),
    ("BrakingRating", "BrakingRating"),
    ("Brand", "Brand"),
    ("BreakableItem", "BreakableItem"),
    ("BreakablePart", "BreakablePart"),
    ("Breaker", "Breaker"),
    ("BreakRoom", "BreakRoom"),
    ("BreedChance", "BreedChance"),
    ("Bridge", "Bridge"),
    ("Buff", "Buff"),
    ("Buffer", "Buffer"),
    ("Build", "Build"),
    ("BuildCost", "BuildCost"),
    ("BuildDuration", "BuildDuration"),
    ("Building", "Building"),
    ("BuildingBonus", "BuildingBonus"),
    ("BuildingCategory", "BuildingCategory"),
    ("BuildingEffect", "BuildingEffect"),
    ("BuildingType", "BuildingType"),
    ("BuildMenuCategory", "BuildMenuCategory"),
    ("BuildTime", "BuildTime"),
    ("BulletSpread", "BulletSpread"),
    ("BulletsToKill", "BulletsToKill"),
    ("Bundle", "Bundle"),
    ("Burst", "Burst"),
    ("BurstDuration", "BurstDuration"),
    ("BurstMass", "BurstMass"),
    ("Business", "Business"),
    ("BusinessActivity", "BusinessActivity"),
    ("BusStop", "BusStop"),
    ("BuyCost", "BuyCost"),
    ("BuyPrice", "BuyPrice"),
    ("Cage", "Cage"),
    ("CageType", "CageType"),
    ("Caliber", "Caliber"),
    ("CaloricValue", "CaloricValue"),
    ("Campaign", "Campaign"),
    ("CannonDamage", "CannonDamage"),
    ("CannonRange", "CannonRange"),
    ("Canon", "Canon"),
    ("Capability", "Capability"),
    ("Capacity", "Capacity"),
    ("Card", "Card"),
    ("CardID", "CardID"),
    ("CardNumber", "CardNumber"),
    ("CardStatus", "CardStatus"),
    ("CardType", "CardType"),
    ("CardUpgrade", "CardUpgrade"),
    ("Career", "Career"),
    ("CareerAvailability", "CareerAvailability"),
    ("CareerModeStatus", "CareerModeStatus"),
    ("Cargo", "Cargo"),
    ("CargoCapacity", "CargoCapacity"),
    ("CargoHoldSize", "CargoHoldSize"),
    ("CargoRank", "CargoRank"),
    ("CargoSlot", "CargoSlot"),
    ("CarManufacturer", "CarManufacturer"),
    ("CarModel", "CarModel"),
    ("Cartridge", "Cartridge"),
    ("CartridgeType", "CartridgeType"),
    ("Case", "Case"),
    ("Casting", "Casting"),
    ("CastingChance", "CastingChance"),
    ("CastingTime", "CastingTime"),
    ("CastTime", "CastTime"),
    ("Catalog", "Catalog"),
    ("CatalogNumber", "CatalogNumber"),
    ("CatchRate", "CatchRate"),
    ("Category", "Category"),
    ("Cause", "Cause"),
    ("CauseOfDeath", "CauseOfDeath"),
    ("CelestialBody", "CelestialBody"),
    ("Cell", "Cell"),
    ("CellSlot", "CellSlot"),
    ("Challenge", "Challenge"),
    ("Chance", "Chance"),
    ("Change", "Change"),
    ("Chaos", "Chaos"),
    ("Chapter", "Chapter"),
    ("ChapterIdentifier", "ChapterIdentifier"),
    ("ChapterNumber", "ChapterNumber"),
    ("Character", "Character"),
    ("CharacterAffiliation", "CharacterAffiliation"),
    ("CharacterAlias", "CharacterAlias"),
    ("CharacterAttribute", "CharacterAttribute"),
    ("CharacterClass", "CharacterClass"),
    ("CharacterDesigner", "CharacterDesigner"),
    ("CharacterEntity", "CharacterEntity"),
    ("CharacterFamily", "CharacterFamily"),
    ("CharacterGroup", "CharacterGroup"),
    ("CharacterIdentity", "CharacterIdentity"),
    ("Characteristic", "Characteristic"),
    ("CharacterLevel", "CharacterLevel"),
    ("CharacterLocation", "CharacterLocation"),
    ("CharacterModel", "CharacterModel"),
    ("CharacterName", "CharacterName"),
    ("CharacterRace", "CharacterRace"),
    ("CharacterRole", "CharacterRole"),
    ("CharacterSpecies", "CharacterSpecies"),
    ("CharacterState", "CharacterState"),
    ("CharacterSubtype", "CharacterSubtype"),
    ("CharacterType", "CharacterType"),
    ("CharacterVariant", "CharacterVariant"),
    ("CharacterVoiceActor", "CharacterVoiceActor"),
    ("Charcoal", "Charcoal"),
    ("Charges", "Charges"),
    ("Charm", "Charm"),
    ("Chemical", "Chemical"),
    ("ChemicalElement", "ChemicalElement"),
    ("ChemicalSymbol", "ChemicalSymbol"),
    ("ChineseHoroscope", "ChineseHoroscope"),
    ("Choice", "Choice"),
    ("Citizenship", "Citizenship"),
    ("City", "City"),
    ("CityArea", "CityArea"),
    ("Civilization", "Civilization"),
    ("CivilizationType", "CivilizationType"),
    ("Clade", "Clade"),
    ("Clan", "Clan"),
    ("Class", "Class"),
    ("Classification", "Classification"),
    ("ClassificationType", "ClassificationType"),
    ("ClassRating", "ClassRating"),
    ("ClassTrait", "ClassTrait"),
    ("CleanlinessRequirement", "CleanlinessRequirement"),
    ("ClearCondition", "ClearCondition"),
    ("Client", "Client"),
    ("Climate", "Climate"),
    ("ClimateType", "ClimateType"),
    ("ClimbingArea", "ClimbingArea"),
    ("ClimbRate", "ClimbRate"),
    ("ClipSize", "ClipSize"),
    ("ClipSizeValue", "ClipSizeValue"),
    ("Clothes", "Clothes"),
    ("Clothing", "Clothing"),
    ("ClothingItem", "ClothingItem"),
    ("ClothingType", "ClothingType"),
    ("ClothPart", "ClothPart"),
    ("Club", "Club"),
    ("Code", "Code"),
    ("Codename", "Codename"),
    ("Coin", "Coin"),
    ("CoinCostDetails", "CoinCostDetails"),
    ("Collectible", "Collectible"),
    ("CollectibleItem", "CollectibleItem"),
    ("CollectibleType", "CollectibleType"),
    ("Collection", "Collection"),
    ("CollectionType", "CollectionType"),
    ("Collective", "Collective"),
    ("ColonizationStrategy", "ColonizationStrategy"),
    ("Colony", "Colony"),
    ("Color", "Color"),
    ("ColorSet", "ColorSet"),
    ("CombatAbility", "CombatAbility"),
    ("CombatAlignment", "CombatAlignment"),
    ("CombatData", "CombatData"),
    ("CombatProficiency", "CombatProficiency"),
    ("CombatRange", "CombatRange"),
    ("CombatStat", "CombatStat"),
    ("CombatStatistic", "CombatStatistic"),
    ("CombatStyle", "CombatStyle"),
    ("CombatType", "CombatType"),
    ("CombatValue", "CombatValue"),
    ("Combo", "Combo"),
    ("Comic", "Comic"),
    ("ComicIssue", "ComicIssue"),
    ("ComicSeries", "ComicSeries"),
    ("Command", "Command"),
    ("Commander", "Commander"),
    ("CommandStructure", "CommandStructure"),
    ("Commission", "Commission"),
    ("CommonName", "CommonName"),
    ("CommunicationStyle", "CommunicationStyle"),
    ("Community", "Community"),
    ("CommunityRelation", "CommunityRelation"),
    ("Company", "Company"),
    ("Compatibility", "Compatibility"),
    ("CompletionCriteria", "CompletionCriteria"),
    ("Complexion", "Complexion"),
    ("Component", "Component"),
    ("Components", "Component"),
    ("ComponentSlot", "ComponentSlot"),
    ("ComponentSlotCount", "ComponentSlotCount"),
    ("ComponentSlots", "ComponentSlot"),
    ("Composer", "Composer"),
    ("Concentration", "Concentration"),
    ("Concept", "Concept"),
    ("Condition", "Condition"),
    ("Conflict", "Conflict"),
    ("ConnectionType", "ConnectionType"),
    ("Consequence", "Consequence"),
    ("Console", "Console"),
    ("ConsoleName", "ConsoleName"),
    ("ConsoleType", "ConsoleType"),
    ("ConstructionSlot", "ConstructionSlot"),
    ("ConsultantUpgrade", "ConsultantUpgrade"),
    ("Consumable", "Consumable"),
    ("ConsumableItem", "ConsumableItem"),
    ("Contact", "Contact"),
    ("Container", "Container"),
    ("Content", "Content"),
    ("ContentAvailability", "ContentAvailability"),
    ("ContentExpansion", "ContentExpansion"),
    ("ContentType", "ContentType"),
    ("Context", "Context"),
    ("Continent", "Continent"),
    ("Continuity", "Continuity"),
    ("ContractLimit", "ContractLimit"),
    ("Contribution", "Contribution"),
    ("ContributionScore", "ContributionScore"),
    ("ContributionValue", "ContributionValue"),
    ("Control", "Control"),
    ("ControlInput", "ControlInput"),
    ("Controller", "Controller"),
    ("ConversationDataID", "ConversationDataID"),
    ("Converter", "Converter"),
    ("CookingMethod", "CookingMethod"),
    ("CookTime", "CookTime"),
    ("Cooldown", "Cooldown"),
    ("CooldownDuration", "CooldownDuration"),
    ("CooldownPeriod", "CooldownPeriod"),
    ("CooldownSpeed", "CooldownSpeed"),
    ("CooldownTime", "CooldownTime"),
    ("Coordinate", "Coordinate"),
    ("Coordinates", "Coordinate"),
    ("Copyright", "Copyright"),
    ("CopyrightHolder", "CopyrightHolder"),
    ("Coromon", "Coromon"),
    ("Corporation", "Corporation"),
    ("Corpse", "Corpse"),
    ("Corruption", "Corruption"),
    ("CorruptionMethod", "CorruptionMethod"),
    ("CorruptionPoints", "CorruptionPoints"),
    ("Cosmetic", "Cosmetic"),
    ("CosmeticItem", "CosmeticItem"),
    ("CosmeticType", "CosmeticType"),
    ("CosmicAlignment", "CosmicAlignment"),
    ("CosmicLaw", "CosmicLaw"),
    ("Cost", "Cost"),
    ("CostDetail", "CostDetail"),
    ("CostType", "CostType"),
    ("CostumePattern", "CostumePattern"),
    ("CostValue", "CostValue"),
    ("Count", "Count"),
    ("Counter", "Counter"),
    ("Country", "Country"),
    ("Course", "Course"),
    ("CoverArt", "CoverArt"),
    ("Coziness", "Coziness"),
    ("Cracker", "Cracker"),
    ("CraftableStatus", "CraftableStatus"),
    ("Crafting", "Crafting"),
    ("CraftingCost", "CraftingCost"),
    ("CraftingInfo", "CraftingInfo"),
    ("CraftingIngredient", "CraftingIngredient"),
    ("CraftingLocation", "CraftingLocation"),
    ("CraftingMaterial", "CraftingMaterial"),
    ("CraftingMethod", "CraftingMethod"),
    ("CraftingProcess", "CraftingProcess"),
    ("CraftingRecipe", "CraftingRecipe"),
    ("CraftingStation", "CraftingStation"),
    ("CraftingStatus", "CraftingStatus"),
    ("CraftingTable", "CraftingTable"),
    ("CraftingTime", "CraftingTime"),
    ("CraftRecipe", "CraftRecipe"),
    ("Crate", "Crate"),
    ("CreationCondition", "CreationCondition"),
    ("CreationTime", "CreationTime"),
    ("Creator", "Creator"),
    ("CreatorTier", "CreatorTier"),
    ("Creature", "Creature"),
    ("CreatureCode", "CreatureCode"),
    ("CreatureType", "CreatureType"),
    ("Credit", "Credit"),
    ("CreditCost", "CreditCost"),
    ("Credits", "Credit"),
    ("Crew", "Crew"),
    ("CrewCount", "CrewCount"),
    ("CrewSize", "CrewSize"),
    ("CrewType", "CrewType"),
    ("Crime", "Crime"),
    ("CrimeScene", "CrimeScene"),
    ("CrimeType", "CrimeType"),
    ("CriminalCapacity", "CriminalCapacity"),
    ("CritChance", "CritChance"),
    ("Criteria", "Criteria"),
    ("Critical", "Critical"),
    ("CriticalChance", "CriticalChance"),
    ("CriticalDamage", "CriticalDamage"),
    ("CriticalDamageValue", "CriticalDamageValue"),
    ("CriticalStatistic", "CriticalStatistic"),
    ("Critter", "Critter"),
    ("Crop", "Crop"),
    ("CScaleRating", "CScaleRating"),
    ("Cult", "Cult"),
    ("Culture", "Culture"),
    ("Cup", "Cup"),
    ("Cure", "Cure"),
    ("Currency", "Currency"),
    ("CurrencyAmount", "CurrencyAmount"),
    ("CurrencyValue", "CurrencyValue"),
    ("Curse", "Curse"),
    ("Customizability", "Customizability"),
    ("Customization", "Customization"),
    ("CustomizationOptions", "CustomizationOptions"),
    ("CustomizationSlot", "CustomizationSlot"),
    ("CutReduction", "CutReduction"),
    ("Cutscene", "Cutscene"),
    ("Damage", "Damage"),
    ("Damageability", "Damageability"),
    ("DamageAttribute", "DamageAttribute"),
    ("DamageAugment", "DamageAugment"),
    ("DamageBonus", "DamageBonus"),
    ("DamageBoost", "DamageBoost"),
    ("DamageCategory", "DamageCategory"),
    ("DamageClassification", "DamageClassification"),
    ("DamageEffect", "DamageEffect"),
    ("DamageElement", "DamageElement"),
    ("DamageModifier", "DamageModifier"),
    ("DamageMultiplier", "DamageMultiplier"),
    ("DamagePenalty", "DamagePenalty"),
    ("DamagePerSecond", "DamagePerSecond"),
    ("DamagePoints", "DamagePoints"),
    ("DamageRange", "DamageRange"),
    ("DamageRating", "DamageRating"),
    ("DamageReduction", "DamageReduction"),
    ("DamageType", "DamageType"),
    ("DamageUpgrade", "DamageUpgrade"),
    ("DamageValue", "DamageValue"),
    ("DangerLevel", "DangerLevel"),
    ("Data", "Data"),
    ("DatabankID", "DatabankID"),
    ("DatabaseEntry", "DatabaseEntry"),
    ("DataCorruption", "DataCorruption"),
    ("DataCorruptionLevel", "DataCorruptionLevel"),
    ("DataCorruptionState", "DataCorruptionState"),
    ("DataCorruptionStatus", "DataCorruptionStatus"),
    ("DataID", "DataID"),
    ("DataIdentifier", "DataIdentifier"),
    ("DataInteger", "DataInteger"),
    ("DataIntegrity", "DataIntegrity"),
    ("DataIntegrityLevel", "DataIntegrityLevel"),
    ("DataIntegrityStatus", "DataIntegrityStatus"),
    ("DataNoneSpecified", "DataNoneSpecified"),
    ("DataNumber", "DataNumber"),
    ("Datapoint", "Datapoint"),
    ("DataProperty", "DataProperty"),
    ("DataQuality", "DataQuality"),
    ("DataState", "DataState"),
    ("DataStatus", "DataStatus"),
    ("Datatype", "Datatype"),
    ("DatatypeProperty", "DatatypeProperty"),
    ("Datatypes", "Datatype"),
    ("DatatypesInteger", "DatatypesInteger"),
    ("DatatypeValue", "DatatypeValue"),
    ("DataValue", "DataValue"),
    ("Date", "Date"),
    ("DateLocation", "DateLocation"),
    ("DateProperty", "DateProperty"),
    ("DateRange", "DateRange"),
    ("DateTime", "DateTime"),
    ("DateValue", "DateValue"),
    ("Day", "Day"),
    ("Death", "Death"),
    ("DeathInfo", "DeathInfo"),
    ("DeathManner", "DeathManner"),
    ("DeathReason", "DeathReason"),
    ("DeathStatus", "DeathStatus"),
    ("Debuff", "Debuff"),
    ("DebugCommand", "DebugCommand"),
    ("Debut", "Debut"),
    ("DebutAppearance", "DebutAppearance"),
    ("DecayRate", "DecayRate"),
    ("Decimal", "Decimal"),
    ("DecimalRange", "DecimalRange"),
    ("DecimalValue", "DecimalValue"),
    ("DeckType", "DeckType"),
    ("Decoration", "Decoration"),
    ("DecreasePerSecond", "DecreasePerSecond"),
    ("DeepSnowHandling", "DeepSnowHandling"),
    ("Defense", "Defense"),
    ("DefenseData", "DefenseData"),
    ("DefenseStat", "DefenseStat"),
    ("DefenseType", "DefenseType"),
    ("DefenseValue", "DefenseValue"),
    ("DefensiveBonus", "DefensiveBonus"),
    ("DegreeMeasurement", "DegreeMeasurement"),
    ("Deity", "Deity"),
    ("DeletionStatus", "DeletionStatus"),
    ("Demo", "Demo"),
    ("DemoBuild", "DemoBuild"),
    ("DemoLevel", "DemoLevel"),
    ("Demon", "Demon"),
    ("Demonym", "Demonym"),
    ("DemoVersion", "DemoVersion"),
    ("Density", "Density"),
    ("Department", "Department"),
    ("Dependency", "Dependency"),
    ("Depth", "Depth"),
    ("DepthRange", "DepthRange"),
    ("DerivationType", "DerivationType"),
    ("Descent", "Descent"),
    ("DescentType", "DescentType"),
    ("Description", "Description"),
    ("DescriptiveString", "DescriptiveString"),
    ("Descriptor", "Descriptor"),
    ("Designation", "Designation"),
    ("Designer", "Designer"),
    ("DesignSource", "DesignSource"),
    ("Desk", "Desk"),
    ("Detail", "Detail"),
    ("DeterorationRate", "DeterorationRate"),
    ("DetonationCondition", "DetonationCondition"),
    ("Developer", "Developer"),
    ("DevelopmentEngine", "DevelopmentEngine"),
    ("DevelopmentRole", "DevelopmentRole"),
    ("DevelopmentStatus", "DevelopmentStatus"),
    ("Deviation", "Deviation"),
    ("Device", "Device"),
    ("DevilFruit", "DevilFruit"),
    ("DevilFruitType", "DevilFruitType"),
    ("DiagnosisRating", "DiagnosisRating"),
    ("Dialogue", "Dialogue"),
    ("DialogueAbility", "DialogueAbility"),
    ("Dice", "Dice"),
    ("Diet", "Diet"),
    ("DietaryHabit", "DietaryHabit"),
    ("DietType", "DietType"),
    ("Difficulty", "Difficulty"),
    ("DifficultyDescription", "DifficultyDescription"),
    ("DifficultyLevel", "DifficultyLevel"),
    ("DifficultyRating", "DifficultyRating"),
    ("DifficultyReward", "DifficultyReward"),
    ("DiffLockType", "DiffLockType"),
    ("Dimension", "Dimension"),
    ("Dimensions", "Dimension"),
    ("DimensionValue", "DimensionValue"),
    ("DiplomaticBenefit", "DiplomaticBenefit"),
    ("Direction", "Direction"),
    ("DirectionType", "DirectionType"),
    ("Director", "Director"),
    ("DirectX", "DirectX"),
    ("Disadvantage", "Disadvantage"),
    ("Disc", "Disc"),
    ("Discipline", "Discipline"),
    ("DiscNumber", "DiscNumber"),
    ("Discount", "Discount"),
    ("DiscoveryInsight", "DiscoveryInsight"),
    ("Discs", "Disc"),
    ("Dish", "Dish"),
    ("Dislike", "Dislike"),
    ("Dislikes", "Dislike"),
    ("Disorder", "Disorder"),
    ("Displacement", "Displacement"),
    ("DisplacementValue", "DisplacementValue"),
    ("Distance", "Distance"),
    ("Distribution", "Distribution"),
    ("DistributionMethod", "DistributionMethod"),
    ("DistributionMetric", "DistributionMetric"),
    ("DistributionPlatform", "DistributionPlatform"),
    ("Distributor", "Distributor"),
    ("District", "District"),
    ("Division", "Division"),
    ("DLC", "DLC"),
    ("Document", "Document"),
    ("Dodge", "Dodge"),
    ("DodgePercent", "DodgePercent"),
    ("Dogtag", "Dogtag"),
    ("Domain", "Domain"),
    ("Door", "Door"),
    ("DoorChoice", "DoorChoice"),
    ("DownloadableContent", "DownloadableContent"),
    ("DPS", "DPS"),
    ("DrawingTime", "DrawingTime"),
    ("Dream", "Dream"),
    ("Drink", "Drink"),
    ("Drive", "Drive"),
    ("Drivetrain", "Drivetrain"),
    ("DrivetrainConfiguration", "DrivetrainConfiguration"),
    ("DrivetrainLayout", "DrivetrainLayout"),
    ("DrivetrainType", "DrivetrainType"),
    ("DriveType", "DriveType"),
    ("Drop", "Drop"),
    ("DropChance", "DropChance"),
    ("DropItem", "DropItem"),
    ("DropMechanism", "DropMechanism"),
    ("DropRates", "DropRates"),
    ("DropValue", "DropValue"),
    ("Dungeon", "Dungeon"),
    ("DungeonSkill", "DungeonSkill"),
    ("DungeonType", "DungeonType"),
    ("Durability", "Durability"),
    ("DurabilityRating", "DurabilityRating"),
    ("Duration", "Duration"),
    ("DurationValue", "DurationValue"),
    ("DurationWithCost", "DurationWithCost"),
    ("DynamicElement", "DynamicElement"),
    ("DynamicFeature", "DynamicFeature"),
    ("DynamicStatus", "DynamicStatus"),
    ("Earnings", "Earnings"),
    ("Eccentricity", "Eccentricity"),
    ("EcologicalInformation", "EcologicalInformation"),
    ("EcologicalRole", "EcologicalRole"),
    ("EcologyRecord", "EcologyRecord"),
    ("Economy", "Economy"),
    ("Edition", "Edition"),
    ("EditionType", "EditionType"),
    ("EditorialNote", "EditorialNote"),
    ("EducationalInstitution", "EducationalInstitution"),
    ("Effect", "Effect"),
    ("EffectDescription", "EffectDescription"),
    ("EffectDetails", "EffectDetails"),
    ("EffectiveRange", "EffectiveRange"),
    ("Effects", "Effect"),
    ("EffectValue", "EffectValue"),
    ("Element", "Element"),
    ("ElementalConversion", "ElementalConversion"),
    ("ElementalEffect", "ElementalEffect"),
    ("ElementalResistance", "ElementalResistance"),
    ("ElementalType", "ElementalType"),
    ("ElementSymbol", "ElementSymbol"),
    ("ElementType", "ElementType"),
    ("Elevation", "Elevation"),
    ("EliteModInfo", "EliteModInfo"),
    ("EliteStatus", "EliteStatus"),
    ("Email", "Email"),
    ("Emblem", "Emblem"),
    ("Emotion", "Emotion"),
    ("Empire", "Empire"),
    ("Employee", "Employee"),
    ("EmployeeCount", "EmployeeCount"),
    ("Ending", "Ending"),
    ("EndingPhrase", "EndingPhrase"),
    ("Endorsement", "Endorsement"),
    ("Enemy", "Enemy"),
    ("EnemyFaction", "EnemyFaction"),
    ("EnemyGroup", "EnemyGroup"),
    ("EnemyTier", "EnemyTier"),
    ("EnemyType", "EnemyType"),
    ("Energy", "Energy"),
    ("EnergyConsumption", "EnergyConsumption"),
    ("EnergyCost", "EnergyCost"),
    ("EnergyGain", "EnergyGain"),
    ("EnergyHex", "EnergyHex"),
    ("EnergyHexes", "EnergyHex"),
    ("EnergyPoints", "EnergyPoints"),
    ("EnergyRegen", "EnergyRegen"),
    ("EnergyRegeneration", "EnergyRegeneration"),
    ("EnergySource", "EnergySource"),
    ("EnergyStation", "EnergyStation"),
    ("EnergyUsage", "EnergyUsage"),
    ("EnergyValue", "EnergyValue"),
    ("Engagement", "Engagement"),
    ("Engine", "Engine"),
    ("EngineDisplacement", "EngineDisplacement"),
    ("EngineLayout", "EngineLayout"),
    ("EngineLife", "EngineLife"),
    ("EngineLocation", "EngineLocation"),
    ("EnginePlacement", "EnginePlacement"),
    ("EngineSpec", "EngineSpec"),
    ("EngineSpecification", "EngineSpecification"),
    ("EngineSpecifications", "EngineSpecification"),
    ("EngineType", "EngineType"),
    ("EnhancedDescription", "EnhancedDescription"),
    ("EnhancedEffect", "EnhancedEffect"),
    ("Enhancement", "Enhancement"),
    ("EntertainmentLabel", "EntertainmentLabel"),
    ("Entity", "Entity"),
    ("EntityCount", "EntityCount"),
    ("EntityForm", "EntityForm"),
    ("EntityID", "EntityID"),
    ("EntitySpawn", "EntitySpawn"),
    ("EntityType", "EntityType"),
    ("Entrance", "Entrance"),
    ("Entry", "Entry"),
    ("Environment", "Environment"),
    ("EnvironmentalCondition", "EnvironmentalCondition"),
    ("EnvironmentalEffect", "EnvironmentalEffect"),
    ("EnvironmentalFeature", "EnvironmentalFeature"),
    ("EnvironmentalHazard", "EnvironmentalHazard"),
    ("EnvironmentCondition", "EnvironmentCondition"),
    ("EnvironmentType", "EnvironmentType"),
    ("Episode", "Episode"),
    ("EpisodeCount", "EpisodeCount"),
    ("EpisodeNumber", "EpisodeNumber"),
    ("EpisodeType", "EpisodeType"),
    ("Epithet", "Epithet"),
    ("Equipment", "Equipment"),
    ("EquipmentOverview", "EquipmentOverview"),
    ("EquipmentSlot", "EquipmentSlot"),
    ("EquipmentType", "EquipmentType"),
    ("EquipSlot", "EquipSlot"),
    ("Era", "Era"),
    ("ESRBRating", "ESRBRating"),
    ("Essence", "Essence"),
    ("EssenceCost", "EssenceCost"),
    ("Establishment", "Establishment"),
    ("EthnicComposition", "EthnicComposition"),
    ("EthnicGroup", "EthnicGroup"),
    ("Ethnicity", "Ethnicity"),
    ("Evade", "Evade"),
    ("Evaluation", "Evaluation"),
    ("EvaluationScore", "EvaluationScore"),
    ("Evasion", "Evasion"),
    ("Event", "Event"),
    ("EventAlias", "EventAlias"),
    ("EventCategory", "EventCategory"),
    ("EventDescription", "EventDescription"),
    ("EventDetails", "EventDetails"),
    ("EventEffect", "EventEffect"),
    ("EventFormat", "EventFormat"),
    ("EventMoment", "EventMoment"),
    ("EventNumber", "EventNumber"),
    ("EventSeries", "EventSeries"),
    ("EventText", "EventText"),
    ("EventTiming", "EventTiming"),
    ("EventTrigger", "EventTrigger"),
    ("EventType", "EventType"),
    ("EvolutionPath", "EvolutionPath"),
    ("ExecutionCapability", "ExecutionCapability"),
    ("ExileDuration", "ExileDuration"),
    ("Expansion", "Expansion"),
    ("ExpansionSet", "ExpansionSet"),
    ("Expedition", "Expedition"),
    ("Experience", "Experience"),
    ("ExperiencePoint", "ExperiencePoint"),
    ("ExperiencePoints", "ExperiencePoint"),
    ("ExperienceType", "ExperienceType"),
    ("Experiment", "Experiment"),
    ("Explosion", "Explosion"),
    ("ExplosionEffect", "ExplosionEffect"),
    ("ExternalService", "ExternalService"),
    ("Extractor", "Extractor"),
    ("ExtractorType", "ExtractorType"),
    ("EyeColor", "EyeColor"),
    ("Eyes", "Eye"),
    ("EyeType", "EyeType"),
    ("FableArt", "FableArt"),
    ("FaceModel", "FaceModel"),
    ("Facility", "Facility"),
    ("FacilitySlot", "FacilitySlot"),
    ("Faction", "Faction"),
    ("FactionRelation", "FactionRelation"),
    ("Factor", "Factor"),
    ("Fame", "Fame"),
    ("Family", "Family"),
    ("FamilyMember", "FamilyMember"),
    ("FamilyRelation", "FamilyRelation"),
    ("FamilyRelationship", "FamilyRelation"),
    ("FAQ", "FAQ"),
    ("Fate", "Fate"),
    ("FateDescription", "FateDescription"),
    ("Favor", "Favor"),
    ("Feature", "Feature"),
    ("FeatureDescription", "FeatureDescription"),
    ("Features", "Feature"),
    ("FeatureType", "FeatureType"),
    ("FeedSystem", "FeedSystem"),
    ("FenceGrade", "FenceGrade"),
    ("Fertilizer", "Fertilizer"),
    ("FictionalOrganization", "FictionalOrganization"),
    ("FieldOfWork", "FieldOfWork"),
    ("Fighter", "Fighter"),
    ("FightingStyle", "FightingStyle"),
    ("File", "File"),
    ("FileName", "FileName"),
    ("FileType", "FileType"),
    ("Film", "Film"),
    ("Filter", "Filter"),
    ("FinalSmash", "FinalSmash"),
    ("FinancialCost", "FinancialCost"),
    ("FinancialInformation", "FinancialInformation"),
    ("FinancialStatus", "FinancialStatus"),
    ("FinancialValue", "FinancialValue"),
    ("Fire", "Fire"),
    ("FireEffect", "FireEffect"),
    ("FireMechanism", "FireMechanism"),
    ("FireMode", "FireMode"),
    ("FireRate", "FireRate"),
    ("FireRateCategory", "FireRateCategory"),
    ("FireRateValue", "FireRateValue"),
    ("FireTime", "FireTime"),
    ("Firewall", "Firewall"),
    ("FirewallLevel", "FirewallLevel"),
    ("FiringMechanism", "FiringMechanism"),
    ("FiringMode", "FiringMode"),
    ("FirstShotMultiplier", "FirstShotMultiplier"),
    ("Fish", "Fish"),
    ("FishingRod", "FishingRod"),
    ("FishSpecies", "FishSpecies"),
    ("Flag", "Flag"),
    ("Flammability", "Flammability"),
    ("Flavor", "Flavor"),
    ("FlavorText", "FlavorText"),
    ("FlinchEffect", "FlinchEffect"),
    ("Float", "Float"),
    ("FloatValue", "FloatValue"),
    ("Floor", "Floor"),
    ("FloorCount", "FloorCount"),
    ("Flower", "Flower"),
    ("Focus", "Focus"),
    ("FocusArea", "FocusArea"),
    ("FocusStat", "FocusStat"),
    ("Foe", "Foe"),
    ("Food", "Food"),
    ("FoodCategory", "FoodCategory"),
    ("FoodCost", "FoodCost"),
    ("FoodItem", "FoodItem"),
    ("FoodSource", "FoodSource"),
    ("FoodType", "FoodType"),
    ("Footprint", "Footprint"),
    ("FootSize", "FootSize"),
    ("Footwear", "Footwear"),
    ("Form", "Form"),
    ("Format", "Format"),
    ("Formation", "Formation"),
    ("FormatType", "FormatType"),
    ("Formula", "Formula"),
    ("FoundationInfo", "FoundationInfo"),
    ("Fraction", "Fraction"),
    ("Franchise", "Franchise"),
    ("Frequency", "Frequency"),
    ("FrequencyType", "FrequencyType"),
    ("Friend", "Friend"),
    ("Fuel", "Fuel"),
    ("FuelParameter", "FuelParameter"),
    ("FuelStation", "FuelStation"),
    ("Function", "Function"),
    ("Functionality", "Functionality"),
    ("FunctionType", "FunctionType"),
    ("Gadget", "Gadget"),
    ("Galaxy", "Galaxy"),
    ("Game", "Game"),
    ("GameAbility", "GameAbility"),
    ("GameAct", "GameAct"),
    ("GameAction", "GameAction"),
    ("GameAlteration", "GameAlteration"),
    ("GameAppearance", "GameAppearance"),
    ("GameArea", "GameArea"),
    ("GameArtist", "GameArtist"),
    ("GameCartridge", "GameCartridge"),
    ("GameCase", "GameCase"),
    ("GameCategory", "GameCategory"),
    ("GameChapter", "GameChapter"),
    ("GameCharacter", "GameCharacter"),
    ("GameClass", "GameClass"),
    ("GameCode", "GameCode"),
    ("GameCommand", "GameCommand"),
    ("GameComposer", "GameComposer"),
    ("GameConsole", "GameConsole"),
    ("GameContent", "GameContent"),
    ("GameContext", "GameContext"),
    ("GameDay", "GameDay"),
    ("GameDesigner", "GameDesigner"),
    ("GameDeveloper", "GameDeveloper"),
    ("GameDevelopmentStudio", "GameDevelopmentStudio"),
    ("GameDimension", "GameDimension"),
    ("GameEdition", "GameEdition"),
    ("GameEffect", "GameEffect"),
    ("GameElement", "GameElement"),
    ("GameEngine", "GameEngine"),
    ("GameEnvironment", "GameEnvironment"),
    ("GameEpisode", "GameEpisode"),
    ("GameEra", "GameEra"),
    ("GameEvent", "GameEvent"),
    ("GameExpansion", "GameExpansion"),
    ("GameFeature", "GameFeature"),
    ("GameFile", "GameFile"),
    ("GameGeneration", "GameGeneration"),
    ("GameGenre", "GameGenre"),
    ("GameImpact", "GameImpact"),
    ("GameItem", "GameItem"),
    ("GameLevel", "GameLevel"),
    ("GameLocation", "GameLocation"),
    ("GameLoop", "GameLoop"),
    ("GameMechanic", "GameMechanic"),
    ("GameMechanicAttribute", "GameMechanicAttribute"),
    ("GameMedia", "GameMedia"),
    ("GameMode", "GameMode"),
    ("GameModeCategory", "GameModeCategory"),
    ("GameModes", "GameMode"),
    ("GameModeType", "GameModeType"),
    ("GameObjective", "GameObjective"),
    ("GamePart", "GamePart"),
    ("Gameplay", "Gameplay"),
    ("GameplayAction", "GameplayAction"),
    ("GameplayAdvice", "GameplayAdvice"),
    ("GameplayAlteration", "GameplayAlteration"),
    ("GameplayDetail", "GameplayDetail"),
    ("GameplayEdition", "GameplayEdition"),
    ("GameplayEffect", "GameplayEffect"),
    ("GamePlayer", "GamePlayer"),
    ("GameplayFeature", "GameplayFeature"),
    ("GameplayFocus", "GameplayFocus"),
    ("GameplayMechanic", "GameplayMechanic"),
    ("GameplayMode", "GameplayMode"),
    ("GameplayRequirement", "GameplayRequirement"),
    ("GameplayType", "GameplayType"),
    ("GameProgress", "GameProgress"),
    ("GamePublisher", "GamePublisher"),
    ("GameResource", "GameResource"),
    ("GameReward", "GameReward"),
    ("GamerScore", "GamerScore"),
    ("GameRundown", "GameRundown"),
    ("GameSection", "GameSection"),
    ("GameSegment", "GameSegment"),
    ("GameSeries", "GameSeries"),
    ("GameState", "GameState"),
    ("GameStudio", "GameStudio"),
    ("GameStyle", "GameStyle"),
    ("GameTitle", "GameTitle"),
    ("GameType", "GameType"),
    ("GameUpdate", "GameUpdate"),
    ("GameVersion", "GameVersion"),
    ("GameVersionType", "GameVersionType"),
    ("GameWorld", "GameWorld"),
    ("GameYear", "GameYear"),
    ("GamingPlatform", "GamingPlatform"),
    ("GamingSkill", "GamingSkill"),
    ("Gang", "Gang"),
    ("Garage", "Garage"),
    ("Gas", "Gas"),
    ("GasCost", "GasCost"),
    ("Gateway", "Gateway"),
    ("GaugeSize", "GaugeSize"),
    ("GDP", "GDP"),
    ("Gems", "Gems"),
    ("Gender", "Gender"),
    ("GenderIdentity", "GenderIdentity"),
    ("GenderType", "GenderType"),
    ("GeneCode", "GeneCode"),
    ("Generation", "Generation"),
    ("GenerationMethod", "GenerationMethod"),
    ("Generator", "Generator"),
    ("Genre", "Genre"),
    ("Genus", "Genus"),
    ("GeographicalArea", "GeographicalArea"),
    ("GeographicalLocation", "GeographicalLocation"),
    ("GeographicalRegion", "GeographicalRegion"),
    ("GeographicArea", "GeographicalArea"),
    ("GeographicLocation", "GeographicalLocation"),
    ("Geography", "Geography"),
    ("GeologicalType", "GeologicalType"),
    ("GeometricShape", "GeometricShape"),
    ("GestationPeriod", "GestationPeriod"),
    ("Ghost", "Ghost"),
    ("GibFactor", "GibFactor"),
    ("Gift", "Gift"),
    ("Glider", "Glider"),
    ("GlyphDerivation", "GlyphDerivation"),
    ("GlyphType", "GlyphType"),
    ("Goal", "Goal"),
    ("Gold", "Gold"),
    ("GoldAmount", "GoldAmount"),
    ("GoldCost", "GoldCost"),
    ("Goods", "Goods"),
    ("GoodsType", "GoodsType"),
    ("Government", "Government"),
    ("GovernmentStructure", "GovernmentStructure"),
    ("GovernmentType", "GovernmentType"),
    ("Grade", "Grade"),
    ("GrammarRule", "GrammarRule"),
    ("GraphicArtist", "GraphicArtist"),
    ("Graphics", "Graphics"),
    ("GravitationalParameter", "GravitationalParameter"),
    ("Gravity", "Gravity"),
    ("GravityType", "GravityType"),
    ("GravityValue", "GravityValue"),
    ("GriefStage", "GriefStage"),
    ("GroundDefense", "GroundDefense"),
    ("Group", "Group"),
    ("GroupSize", "GroupSize"),
    ("Growth", "Growth"),
    ("Guard", "Guard"),
    ("GuardianSpirit", "GuardianSpirit"),
    ("GuardingProficiency", "GuardingProficiency"),
    ("GUID", "GUID"),
    ("Habit", "Habit"),
    ("Habitat", "Habitat"),
    ("HabitatArea", "HabitatArea"),
    ("Hair", "Hair"),
    ("HairColor", "HairColor"),
    ("HairStyle", "HairStyle"),
    ("HairType", "HairType"),
    ("Handle", "Handle"),
    ("Handling", "Handling"),
    ("HandlingDescription", "HandlingDescription"),
    ("HandlingRating", "HandlingRating"),
    ("HandlingStyle", "HandlingStyle"),
    ("Hands", "Hand"),
    ("Happiness", "Happiness"),
    ("HardAttack", "HardAttack"),
    ("HardpointCount", "HardpointCount"),
    ("Harvester", "Harvester"),
    ("HarvestingCondition", "HarvestingCondition"),
    ("HarvestingNode", "HarvestingNode"),
    ("Hash", "Hash"),
    ("HatCart", "HatCart"),
    ("Hazard", "Hazard"),
    ("HazardLevel", "HazardLevel"),
    ("HazardType", "HazardType"),
    ("HeadDirection", "HeadDirection"),
    ("Headgear", "Headgear"),
    ("HeadType", "HeadType"),
    ("Health", "Health"),
    ("HealthBonus", "HealthBonus"),
    ("HealthDrop", "HealthDrop"),
    ("HealthForm", "HealthForm"),
    ("HealthLevel", "HealthLevel"),
    ("HealthModifier", "HealthModifier"),
    ("HealthPoint", "HealthPoint"),
    ("HealthPoints", "HealthPoint"),
    ("HealthRecovery", "HealthRecovery"),
    ("HealthRegen", "HealthRegen"),
    ("HealthStat", "HealthStat"),
    ("HealthStatus", "HealthStatus"),
    ("HealthUpgrade", "HealthUpgrade"),
    ("HealthValue", "HealthValue"),
    ("Heat", "Heat"),
    ("HeatingLevel", "HeatingLevel"),
    ("HeatValue", "HeatValue"),
    ("Height", "Height"),
    ("HeightMeasurement", "HeightMeasurement"),
    ("Heritage", "Heritage"),
    ("Hero", "Hero"),
    ("HeroClass", "HeroClass"),
    ("HeroType", "HeroType"),
    ("Hex", "Hex"),
    ("HexMeasurement", "HexMeasurement"),
    ("HexSize", "HexSize"),
    ("Highlight", "Highlight"),
    ("HistoricalEvent", "HistoricalEvent"),
    ("HistoricalFact", "HistoricalFact"),
    ("HistoricalTrait", "HistoricalTrait"),
    ("History", "History"),
    ("Hitbox", "Hitbox"),
    ("HitboxType", "HitboxType"),
    ("HitCount", "HitCount"),
    ("HitPoints", "HitPoint"),
    ("HitType", "HitType"),
    ("Hobby", "Hobby"),
    ("Hold", "Hold"),
    ("HoldSize", "HoldSize"),
    ("HolsteringTime", "HolsteringTime"),
    ("HomeRequest", "HomeRequest"),
    ("HomeType", "HomeType"),
    ("Horoscope", "Horoscope"),
    ("House", "House"),
    ("HouseLayout", "HouseLayout"),
    ("HP", "HP"),
    ("HP5", "HP5"),
    ("HPRecovery", "HPRecovery"),
    ("HPRegen", "HPRegen"),
    ("HPRegeneration", "HPRegeneration"),
    ("Hull", "Hull"),
    ("HullClass", "HullClass"),
    ("HullClassType", "HullClassType"),
    ("HullStrength", "HullStrength"),
    ("HullType", "HullType"),
    ("HungerEffect", "HungerEffect"),
    ("Hyperdrive", "Hyperdrive"),
    ("HyperdriveType", "HyperdriveType"),
    ("Icon", "Icon"),
    ("ID", "ID"),
    ("Identification", "Identification"),
    ("Identifier", "Identifier"),
    ("Identity", "Identity"),
    ("Ideology", "Ideology"),
    ("Illustrator", "Illustrator"),
    ("Image", "Image"),
    ("ImageCollection", "ImageCollection"),
    ("ImageFile", "ImageFile"),
    ("Immunity", "Immunity"),
    ("ImmunityType", "ImmunityType"),
    ("Impact", "Impact"),
    ("ImpactLevel", "ImpactLevel"),
    ("Importance", "Importance"),
    ("ImprecisionValue", "ImprecisionValue"),
    ("ImprovementAction", "ImprovementAction"),
    ("Income", "Income"),
    ("IncomeRule", "IncomeRule"),
    ("Index", "Index"),
    ("IndexNumber", "IndexNumber"),
    ("Individual", "Individual"),
    ("Industry", "Industry"),
    ("IndustryRichness", "IndustryRichness"),
    ("IndustryStatus", "IndustryStatus"),
    ("IndustryType", "IndustryType"),
    ("Infection", "Infection"),
    ("InfectionStatus", "InfectionStatus"),
    ("Influence", "Influence"),
    ("InfluenceArea", "InfluenceArea"),
    ("InfluenceValue", "InfluenceValue"),
    ("Information", "Information"),
    ("Infrastructure", "Infrastructure"),
    ("InfrastructureRequirement", "InfrastructureRequirement"),
    ("Ingredient", "Ingredient"),
    ("Inhabitant", "Inhabitant"),
    ("InhabitantStatus", "InhabitantStatus"),
    ("Initiative", "Initiative"),
    ("InitiativeValue", "InitiativeValue"),
    ("InjuredCapacity", "InjuredCapacity"),
    ("InjuredCount", "InjuredCount"),
    ("InkConsumption", "InkConsumption"),
    ("InkConsumptionValue", "InkConsumptionValue"),
    ("InkUsage", "InkUsage"),
    ("Input", "Input"),
    ("InputCombination", "InputCombination"),
    ("InputCommand", "InputCommand"),
    ("InputDevice", "InputDevice"),
    ("InputKey", "InputKey"),
    ("InputMethod", "InputMethod"),
    ("InputSequence", "InputSequence"),
    ("Insight", "Insight"),
    ("InspectedText", "InspectedText"),
    ("InspectRemarks", "InspectRemarks"),
    ("InspectText", "InspectText"),
    ("Institution", "Institution"),
    ("Instructor", "Instructor"),
    ("Instrument", "Instrument"),
    ("Int", "Int"),
    ("Integer", "Integer"),
    ("IntegerRange", "IntegerRange"),
    ("IntegerValue", "IntegerValue"),
    ("Intellect", "Intellect"),
    ("Intelligence", "Intelligence"),
    ("IntelligenceLevel", "IntelligenceLevel"),
    ("IntelligenceType", "IntelligenceType"),
    ("Interaction", "Interaction"),
    ("Interactive", "Interactive"),
    ("InteractiveObject", "InteractiveObject"),
    ("InteractivityLevel", "InteractivityLevel"),
    ("Interceptor", "Interceptor"),
    ("Interest", "Interest"),
    ("InteriorFeature", "InteriorFeature"),
    ("InternalArmorID", "InternalArmorID"),
    ("InternalID", "InternalID"),
    ("InternalName", "InternalName"),
    ("InternetResource", "InternetResource"),
    ("Interval", "Interval"),
    ("IntimidationLevel", "IntimidationLevel"),
    ("IntroductionDate", "IntroductionDate"),
    ("IntroductionPatch", "IntroductionPatch"),
    ("Inventory", "Inventory"),
    ("InventoryCategory", "InventoryCategory"),
    ("InventorySlot", "InventorySlot"),
    ("InventorySpace", "InventorySpace"),
    ("IPAddress", "IPAddress"),
    ("ISBN", "ISBN"),
    ("island", "Island"),
    ("Island", "Island"),
    ("item", "Item"),
    ("Item", "Item"),
    ("ItemAction", "ItemAction"),
    ("Item Action", "ItemAction"),
    ("ItemAlias", "ItemAlias"),
    ("item bundle", "ItemBundle"),
    ("ItemBundle", "ItemBundle"),
    ("item category", "ItemCategory"),
    ("ItemCategory", "ItemCategory"),
    ("Item Category", "ItemCategory"),
    ("ItemClass", "ItemClass"),
    ("Item Class", "ItemClass"),
    ("ItemDescription", "ItemDescription"),
    ("Item Effect", "ItemEffect"),
    ("item group", "ItemGroup"),
    ("ItemGroup", "ItemGroup"),
    ("Item Group", "ItemGroup"),
    ("ItemHash", "ItemHash"),
    ("ItemID", "ItemID"),
    ("Item ID", "ItemID"),
    ("ItemLabel", "ItemLabel"),
    ("ItemLevel", "ItemLevel"),
    ("Item Level", "ItemLevel"),
    ("ItemPack", "ItemPack"),
    ("Item Pack", "ItemPack"),
    ("Item Rarity", "ItemRarity"),
    ("item requirement", "ItemRequirement"),
    ("Items", "Item"),
    ("ItemSet", "ItemSet"),
    ("ItemSlot", "ItemSlot"),
    ("Item Slot", "ItemSlot"),
    ("ItemSource", "ItemSource"),
    ("Item Status", "ItemStatus"),
    ("item type", "ItemType"),
    ("ItemType", "ItemType"),
    ("Item Type", "ItemType"),
    ("ItemUsage", "ItemUsage"),
    ("item use", "ItemUse"),
    ("ItemWeight", "ItemWeight"),
    ("Job", "Job"),
    ("JobPosition", "JobPosition"),
    ("JobStatus", "JobStatus"),
    ("JobTitle", "JobTitle"),
    ("Join Date", "JoinDate"),
    ("Jump", "Jump"),
    ("JumpPoint", "JumpPoint"),
    ("JungleType", "JungleType"),
    ("KanjiCharacter", "KanjiCharacter"),
    ("kart", "Kart"),
    ("Kart", "Kart"),
    ("Keepsake", "Keepsake"),
    ("Key", "Key"),
    ("KeyBinding", "KeyBinding"),
    ("KeyboardShortcut", "KeyboardShortcut"),
    ("Keyboard Shortcut", "KeyboardShortcut"),
    ("keyword", "Keyword"),
    ("Keyword", "Keyword"),
    ("Keywords", "Keyword"),
    ("Ki", "Ki"),
    ("KillCount", "KillCount"),
    ("KillType", "KillType"),
    ("Kill Type", "KillType"),
    ("Kin", "Kin"),
    ("Kingdom", "Kingdom"),
    ("Kit", "Kit"),
    ("KiValue", "KiValue"),
    ("Knockback", "Knockback"),
    ("KnockbackEffect", "KnockbackEffect"),
    ("Knockback Level", "KnockbackLevel"),
    ("KnockbackValue", "KnockbackValue"),
    ("Knowledge", "Knowledge"),
    ("kodama", "Kodama"),
    ("Kodama", "Kodama"),
    ("label", "Label"),
    ("Label", "Label"),
    ("Labyrinth", "Labyrinth"),
    ("LandArea", "LandArea"),
    ("Land Area", "LandArea"),
    ("Landmark", "Landmark"),
    ("language", "Language"),
    ("Language", "Language"),
    ("LanguageRepresentation", "LanguageRepresentation"),
    ("LanguageStructure", "LanguageStructure"),
    ("Laptop", "Laptop"),
    ("Laterality", "Laterality"),
    ("launch", "Launch"),
    ("Launch Rating", "LaunchRating"),
    ("Law", "Law"),
    ("LawStatus", "LawStatus"),
    ("Layout", "Layout"),
    ("Layout Specification", "LayoutSpecification"),
    ("LayoutType", "LayoutType"),
    ("Layout Type", "LayoutType"),
    ("Leader", "Leader"),
    ("LeaderEffect", "LeaderEffect"),
    ("Leadership", "Leadership"),
    ("LeaderType", "LeaderType"),
    ("Learning Level", "LearningLevel"),
    ("Learning Method", "LearningMethod"),
    ("Learning Type", "LearningType"),
    ("Learn Level", "LearnLevel"),
    ("Learn Type", "LearnType"),
    ("Legend Progression", "LegendProgression"),
    ("LegendType", "LegendType"),
    ("length", "Length"),
    ("Length", "Length"),
    ("LengthMeasurement", "LengthMeasurement"),
    ("LengthType", "LengthType"),
    ("LethalThrowCapability", "LethalThrowCapability"),
    ("Letter", "Letter"),
    ("level", "Level"),
    ("Level", "Level"),
    ("LevelBonus", "LevelBonus"),
    ("LevelCode", "LevelCode"),
    ("Level Effect", "LevelEffect"),
    ("LevelFeature", "LevelFeature"),
    ("LevelID", "LevelID"),
    ("LevelNumber", "LevelNumber"),
    ("Level Number", "LevelNumber"),
    ("level range", "LevelRange"),
    ("LevelRange", "LevelRange"),
    ("Level Range", "LevelRange"),
    ("LevelRequirement", "LevelRequirement"),
    ("Level Requirement", "LevelRequirement"),
    ("Level Scaling", "LevelScaling"),
    ("LevelType", "LevelType"),
    ("LevelUpBonus", "LevelUpBonus"),
    ("LevelUpStat", "LevelUpStat"),
    ("Licensee", "Licensee"),
    ("Life", "Life"),
    ("Life Capsule", "LifeCapsule"),
    ("LifeClass", "LifeClass"),
    ("LifeCount", "LifeCount"),
    ("LifeExpectancy", "LifeExpectancy"),
    ("Lifespan", "Lifespan"),
    ("LifeSteal", "LifeSteal"),
    ("LifeType", "LifeType"),
    ("Like", "Like"),
    ("Likes", "Like"),
    ("Limit", "Limit"),
    ("Limitation", "Limitation"),
    ("limitations", "Limitation"),
    ("Limitations", "Limitation"),
    ("Limiter", "Limiter"),
    ("LimitValue", "LimitValue"),
    ("LinguisticElement", "LinguisticElement"),
    ("link", "Link"),
    ("Link", "Link"),
    ("Liquid", "Liquid"),
    ("List", "List"),
    ("Listing", "Listing"),
    ("List of Numbers", "ListOfNumbers"),
    ("literal", "Literal"),
    ("Literal", "Literal"),
    ("Literal (Numeric)", "LiteralNumeric"),
    ("LiteraryWork", "LiteraryWork"),
    ("Literature", "Literature"),
    ("Loadout", "Loadout"),
    ("location", "Location"),
    ("Location", "Location"),
    ("Location/Character", "LocationCharacter"),
    ("LocationDescription", "LocationDescription"),
    ("LocationDetail", "LocationDetail"),
    ("LocationDirection", "LocationDirection"),
    ("LocationFeature", "LocationFeature"),
    ("LocationName", "LocationName"),
    ("LocationRegion", "LocationRegion"),
    ("Location, Resource", "LocationResource"),
    ("LocationSector", "LocationSector"),
    ("LocationSequence", "LocationSequence"),
    ("LocationSource", "LocationSource"),
    ("LocationStatus", "LocationStatus"),
    ("LocationTime", "LocationTime"),
    ("LocationType", "LocationType"),
    ("Loop", "Loop"),
    ("Loot", "Loot"),
    ("LootAvailability", "LootAvailability"),
    ("LootItem", "LootItem"),
    ("LootStatus", "LootStatus"),
    ("LootTier", "LootTier"),
    ("Lore", "Lore"),
    ("LoreEntry", "LoreEntry"),
    ("LoreFileNumber", "LoreFileNumber"),
    ("LoreType", "LoreType"),
    ("LostVehicle", "LostVehicle"),
    ("LuminousLevel", "LuminousLevel"),
    ("Lyricist", "Lyricist"),
    ("Machine", "Machine"),
    ("Magazine", "Magazine"),
    ("MagazineCapacity", "MagazineCapacity"),
    ("MagazineSize", "MagazineSize"),
    ("Magic", "Magic"),
    ("MagicalAbility", "MagicalAbility"),
    ("MagicalProtection", "MagicalProtection"),
    ("MagicCost", "MagicCost"),
    ("MagicDefense", "MagicDefense"),
    ("MagicDefenseValue", "MagicDefenseValue"),
    ("MagicLifeSteal", "MagicLifeSteal"),
    ("MagicPierce", "MagicPierce"),
    ("MagicPoints", "MagicPoints"),
    ("MagicPower", "MagicPower"),
    ("MagicPowerStat", "MagicPowerStat"),
    ("MagicPowerValue", "MagicPowerValue"),
    ("MagicSchool", "MagicSchool"),
    ("MagicType", "MagicType"),
    ("MagicUpgrade", "MagicUpgrade"),
    ("Magnification", "Magnification"),
    ("MainframeExpansion", "MainframeExpansion"),
    ("Maintenance", "Maintenance"),
    ("MaintenanceRequirement", "MaintenanceRequirement"),
    ("Make", "Make"),
    ("Maker", "Maker"),
    ("Mana", "Mana"),
    ("ManaCost", "ManaCost"),
    ("Manager", "Manager"),
    ("ManaPoints", "ManaPoints"),
    ("ManaRecovery", "ManaRecovery"),
    ("ManaRegen", "ManaRegen"),
    ("ManaRegeneration", "ManaRegeneration"),
    ("ManaType", "ManaType"),
    ("ManaValue", "ManaValue"),
    ("Maneuverability", "Maneuverability"),
    ("ManeuverabilityRating", "ManeuverabilityRating"),
    ("ManeuverabilityType", "ManeuverabilityType"),
    ("Manga", "Manga"),
    ("MangaChapter", "MangaChapter"),
    ("MangaEvent", "MangaEvent"),
    ("Manufacturer", "Manufacturer"),
    ("ManufacturingFacility", "ManufacturingFacility"),
    ("Map", "Map"),
    ("MapCoordinates", "MapCoordinates"),
    ("MapFile", "MapFile"),
    ("MapSeries", "MapSeries"),
    ("MapSize", "MapSize"),
    ("MapType", "MapType"),
    ("MapVariant", "MapVariant"),
    ("MarineClass", "MarineClass"),
    ("MaritalStatus", "MaritalStatus"),
    ("Market", "Market"),
    ("Marketability", "Marketability"),
    ("MarketabilityStatus", "MarketabilityStatus"),
    ("MarketStatus", "MarketStatus"),
    ("MarketType", "MarketType"),
    ("MarkingFlag", "MarkingFlag"),
    ("Mass", "Mass"),
    ("MassFlowRate", "MassFlowRate"),
    ("MassRate", "MassRate"),
    ("Mastery", "Mastery"),
    ("MatchType", "MatchType"),
    ("Material", "Material"),
    ("MaterialAmount", "MaterialAmount"),
    ("MaterialCost", "MaterialCost"),
    ("MaterialSuffix", "MaterialSuffix"),
    ("MatterGun", "MatterGun"),
    ("MaxAttachments", "MaxAttachments"),
    ("MaxCargoRank", "MaxCargoRank"),
    ("MaxDamage", "MaxDamage"),
    ("MaximumAmmunition", "MaximumAmmunition"),
    ("MaximumBonds", "MaximumBonds"),
    ("MaximumRange", "MaximumRange"),
    ("MaxPowerplantRank", "MaxPowerplantRank"),
    ("MaxShieldRank", "MaxShieldRank"),
    ("MaxSpeed", "MaxSpeed"),
    ("MaxStock", "MaxStock"),
    ("MeansOfTravel", "MeansOfTravel"),
    ("Measure", "Measure"),
    ("Measurement", "Measurement"),
    ("Mech", "Mech"),
    ("Mecha", "Mecha"),
    ("Mechanic", "Mechanic"),
    ("MechanicType", "MechanicType"),
    ("Mechanism", "Mechanism"),
    ("MechanismDescription", "MechanismDescription"),
    ("MechanismType", "MechanismType"),
    ("MechClass", "MechClass"),
    ("Medal", "Medal"),
    ("Media", "Media"),
    ("MediaAppearance", "MediaAppearance"),
    ("MediaDistribution", "MediaDistribution"),
    ("MediaFormat", "MediaFormat"),
    ("MediaLink", "MediaLink"),
    ("MediaProduction", "MediaProduction"),
    ("MediaTitle", "MediaTitle"),
    ("MediaType", "MediaType"),
    ("MeditationSpot", "MeditationSpot"),
    ("Medium", "Medium"),
    ("MeleeDamage", "MeleeDamage"),
    ("MeleeOffence", "MeleeOffence"),
    ("Member", "Member"),
    ("Memory", "Memory"),
    ("Menu", "Menu"),
    ("MenuCategory", "MenuCategory"),
    ("MenuItem", "MenuItem"),
    ("Message", "Message"),
    ("MetaCategory", "MetaCategory"),
    ("Metadata", "Metadata"),
    ("MetadataID", "MetadataID"),
    ("Method", "Method"),
    ("MetroStation", "MetroStation"),
    ("Might", "Might"),
    ("Milestone", "Milestone"),
    ("MilestoneSet", "MilestoneSet"),
    ("Military", "Military"),
    ("MilitaryBranch", "MilitaryBranch"),
    ("MilitaryCorp", "MilitaryCorp"),
    ("MilitaryDivision", "MilitaryDivision"),
    ("MilitaryFaction", "MilitaryFaction"),
    ("MilitaryForce", "MilitaryForce"),
    ("MilitaryFormation", "MilitaryFormation"),
    ("MilitaryLeader", "MilitaryLeader"),
    ("MilitaryOrganization", "MilitaryOrganization"),
    ("MilitaryRole", "MilitaryRole"),
    ("MilitaryService", "MilitaryService"),
    ("MilitaryType", "MilitaryType"),
    ("MilitaryUnit", "MilitaryUnit"),
    ("MilitaryUnitType", "MilitaryUnitType"),
    ("Mind", "Mind"),
    ("MiniBoss", "MiniBoss"),
    ("Minigame", "Minigame"),
    ("Minion", "Minion"),
    ("Minute", "Minute"),
    ("Miscellaneous", "Miscellaneous"),
    ("Missile", "Missile"),
    ("MissileDamage", "MissileDamage"),
    ("MissileHardpoint", "MissileHardpoint"),
    ("MissileHardpointCount", "MissileHardpointCount"),
    ("MissileHardpoints", "MissileHardpoint"),
    ("MissileRange", "MissileRange"),
    ("Mission", "Mission"),
    ("MissionCategory", "MissionCategory"),
    ("MissionCode", "MissionCode"),
    ("MissionCount", "MissionCount"),
    ("MissionIdentifier", "MissionIdentifier"),
    ("MissionLength", "MissionLength"),
    ("MissionLevel", "MissionLevel"),
    ("MissionNumber", "MissionNumber"),
    ("MissionObjective", "MissionObjective"),
    ("MissionRequirement", "MissionRequirement"),
    ("MissionType", "MissionType"),
    ("Mobility", "Mobility"),
    ("MobilityClass", "MobilityClass"),
    ("MobilityRating", "MobilityRating"),
    ("MobilityScore", "MobilityScore"),
    ("MobilityStat", "MobilityStat"),
    ("Mod", "Mod"),
    ("ModCapacity", "ModCapacity"),
    ("ModCapacityValue", "ModCapacityValue"),
    ("ModCount", "ModCount"),
    ("Mode", "Mode"),
    ("Model", "Model"),
    ("ModeLocation", "ModeLocation"),
    ("ModelYear", "ModelYear"),
    ("ModeType", "ModeType"),
    ("Modification", "Modification"),
    ("ModificationCapacity", "ModificationCapacity"),
    ("ModificationLevels", "ModificationLevels"),
    ("Modifier", "Modifier"),
    ("Modifiers", "Modifier"),
    ("ModStatus", "ModStatus"),
    ("Module", "Module"),
    ("ModuleLimit", "ModuleLimit"),
    ("ModuleSlot", "ModuleSlot"),
    ("ModuleSlots", "ModuleSlot"),
    ("MonetaryAmount", "MonetaryAmount"),
    ("MonetaryValue", "MonetaryValue"),
    ("Money", "Money"),
    ("MoneyValue", "MoneyValue"),
    ("Monster", "Monster"),
    ("MonsterClass", "MonsterClass"),
    ("MonsterType", "MonsterType"),
    ("MonsterVariant", "MonsterVariant"),
    ("Mood", "Mood"),
    ("MoodPerk", "MoodPerk"),
    ("Morale", "Morale"),
    ("Morality", "Morality"),
    ("MorselUnit", "MorselUnit"),
    ("Motive", "Motive"),
    ("Motto", "Motto"),
    ("MountingType", "MountingType"),
    ("MountPoint", "MountPoint"),
    ("MountPointCount", "MountPointCount"),
    ("MountPoints", "MountPoint"),
    ("Move", "Move"),
    ("Movement", "Movement"),
    ("MovementBehavior", "MovementBehavior"),
    ("MovementEffect", "MovementEffect"),
    ("MovementPattern", "MovementPattern"),
    ("MovementSpeed", "MovementSpeed"),
    ("MovementSpeedStat", "MovementSpeedStat"),
    ("MovementStat", "MovementStat"),
    ("MovementType", "MovementType"),
    ("MovementValue", "MovementValue"),
    ("Movie", "Movie"),
    ("MP", "MP"),
    ("MP5", "MP5"),
    ("MPRegen", "MPRegen"),
    ("Multiball", "Multiball"),
    ("MultiballMode", "MultiballMode"),
    ("MultiplayerSupport", "MultiplayerSupport"),
    ("Multiplier", "Multiplier"),
    ("MunitionSlot", "MunitionSlot"),
    ("MunitionSlots", "MunitionSlot"),
    ("Music", "Music"),
    ("MusicalArtist", "MusicalArtist"),
    ("MusicalComposer", "MusicalComposer"),
    ("MusicalEntity", "MusicalEntity"),
    ("MusicalGroup", "MusicalGroup"),
    ("MusicalKey", "MusicalKey"),
    ("MusicArtist", "MusicArtist"),
    ("MusicBlock", "MusicBlock"),
    ("MusicComposer", "MusicComposer"),
    ("MusicGenre", "MusicGenre"),
    ("MusicGroup", "MusicGroup"),
    ("Musician", "Musician"),
    ("MusicInformation", "MusicInformation"),
    ("MusicLabel", "MusicLabel"),
    ("MusicPack", "MusicPack"),
    ("MusicPiece", "MusicPiece"),
    ("MusicStems", "MusicStems"),
    ("MusicTheme", "MusicTheme"),
    ("MusicTrack", "MusicTrack"),
    ("MuzzleVelocity", "MuzzleVelocity"),
    ("Mystery", "Mystery"),
    ("Mythology", "Mythology"),
    ("NA", "NA"),
    ("Name", "Name"),
    ("Narration", "Narration"),
    ("Narrative", "Narrative"),
    ("NarrativeObject", "NarrativeObject"),
    ("NarrativeSource", "NarrativeSource"),
    ("Nation", "Nation"),
    ("Nationality", "Nationality"),
    ("NavalAttack", "NavalAttack"),
    ("NavigationRequirement", "NavigationRequirement"),
    ("Negation", "Negation"),
    ("NegationRule", "NegationRule"),
    ("NegationStructure", "NegationStructure"),
    ("NegationType", "NegationType"),
    ("NegativeTrait", "NegativeTrait"),
    ("NegotiationStyle", "NegotiationStyle"),
    ("Network", "Network"),
    ("Newspaper", "Newspaper"),
    ("Nickname", "Nickname"),
    ("Node", "Node"),
    ("NodeID", "NodeID"),
    ("Noise", "Noise"),
    ("NoiseLevel", "NoiseLevel"),
    ("None", "None"),
    ("Note", "Note"),
    ("Notes", "Note")
]

In [53]:
len(output)

1777

In [78]:
# Create classes

for o in tqdm(output):
    cursor.execute("SELECT id FROM fandom_classes WHERE class = %s", (o[1],))
    class_id = cursor.fetchone()
    if class_id is None:
        cursor.execute("INSERT INTO fandom_classes (class) VALUES (%s)", (o[1],))
        conn.commit()
    if o[0] != o[1]:
        cursor.execute("UPDATE fandom_properties_clean SET target_class = %s WHERE target_class = %s", (o[1], o[0]))
        # if cursor.rowcount > 0:
        #     print(f"NUmber of rows affected: {cursor.rowcount}")
        conn.commit()



In [55]:
# Update subclass of:

for o in output:
    if o[2] is not None:
        cursor.execute("SELECT id FROM fandom_classes WHERE class = %s", (o[2],))
        class_id = cursor.fetchone()
        if class_id is not None:
            cursor.execute("UPDATE fandom_classes SET subclassof = %s WHERE class = %s", (class_id[0], o[1]))
            conn.commit()


In [39]:
# conn.rollback()

### Clean up property target classes

In [1]:
template_string_classes_for_properties = """
You are a highly skilled OWL (Web Ontology Language) ontology engineer. Your task is to assist in the creation, validation, and optimization of OWL ontologies, which are formal representations of knowledge. You have expert knowledge in knowledge engineering, description logic, and semantic web technologies. You also excel at defining classes, properties, and relationships between entities, ensuring logical consistency, and facilitating the sharing of knowledge across different domains.

An OWL ontology is a structured framework used to represent and share knowledge about a particular domain. It consists of classes (concepts), properties (relationships and attributes), and individuals (instances). OWL ontologies allow for the modeling of rich, complex relationships between data in a machine-readable format, enabling advanced reasoning, querying, and inference over that data.
Core OWL Concepts:

    Classes: These represent sets or collections of individuals, typically abstract concepts or types (e.g., "Character," "Weapon," "GameLevel").
    Individuals: Instances of classes (e.g., "Mario" is an individual of the class "Character"; "Sword of Flames" is an individual of the class "Weapon").
    Object Properties: Define relationships between two individuals (e.g., "wieldsWeapon" linking a character to a weapon they use, or "locatedIn" linking a character to a particular game level).
    Datatype Properties: Define relationships between an individual and a data value (e.g., "hasHealthPoints" linking a character to a numeric value representing their health).
    SubClassOf: A relation where one class is a subclass of another, inheriting properties (e.g., "BossCharacter" is a subclass of "Character").
    Equivalence: Used to state that two classes or properties are equivalent (e.g., "MagicWeapon" may be declared equivalent to "SpecialWeapon").
    Disjoint Classes: These are classes that cannot share instances (e.g., "Weapon" and "ConsumableItem" are disjoint classes, meaning an item cannot be both a weapon and a consumable).

OWL Inference and Reasoning:

One of the powerful aspects of OWL is that it allows for reasoning over data. Inference engines can deduce new facts based on the relationships and properties defined in the ontology. For example, if "BossCharacter" is a subclass of "Character" and "Bowser" is an individual of "BossCharacter," it can be inferred that "Bowser" is also an individual of "Character." Additionally, if a property like "wieldsWeapon" is defined, you could infer that "Bowser wields a FireballWeapon" if such an individual and relationship are defined.

Your role is organize and structure a list of classes derived from web data. Classes might be the same but have a slightly different spelling, capitalization, wording, abbreviation or plurality. First, you need to clean up these classes and provide a list of final classes. The final classes should be unique.

Second, you need merge some classes in a single class if that makes ontological sense. For example, GameLevel and Level are the same class and should be merged into a single class. All of the class that have In-Game- , or similar, prefix should drop the prefix and be merged with the class without the prefix.

Third, you need to map which classes are subclasses of other classes. For example, BossCharacter is a subclass of Character. 

You are given a list of classes as input. You need to provide a list of tuples as output: ("original class name", "cleaned up class name", "superclass of the class if it exists in the list of classes" or None). For example, if the input is ["Apple", "apple", "Banana", "Ba-Na-Na", "Fruit"], the output should be [("Apple", "Apple", "Fruit"), ("apple", "Apple", "Fruit"), ("Banana", "Banana", "Fruit"), ("Ba-Na-Na", "Banana", "Fruit"), ("Fruit", "Fruit", None)].

Original classes are case-insensitive. Cleaned up classes should be capitalized and have no special characters or spaces.

Input classes: ```{classes}```

"""